In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [3]:
# Load API Credentials
with open ('/Users/jhugh/.secret/yelp_api.json') as f:
    login = json.load(f)
    
#Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api_key'], timeout_s=5.0)

In [4]:
# set our API call parameters
LOCATION = 'Orlando, FL'
TERM = 'Seafood'

In [5]:
#Specifying JSON_FILE filename
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_FL_seafood.json"
JSON_FILE

'Data/results_in_progress_FL_seafood.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_FL_seafood.json already exists.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 20 previous results found.


In [8]:
#Use our yelp_api variable's search)query method
results = yelp_api.search_query(term =TERM,location=LOCATION,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
total_results = results['total']
total_results

1600

In [10]:
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
import time, math
n_pages = math.ceil((results['total']-n_results)/results_per_page)
n_pages

79

In [12]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/79 [00:00<?, ?it/s]

In [15]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6slmz-P3XOWUq6DqN7Ujsw,happy-snapper-seafood-restaurant-orlando,Happy Snapper Seafood Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/KMLWln...,False,https://www.yelp.com/biz/happy-snapper-seafood...,366,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",5.0,"{'latitude': 28.468527, 'longitude': -81.367425}","[pickup, delivery]",$$,"{'address1': '6327 S Orange Ave', 'address2': ...",+14077570976,(407) 757-0976,3872.699824
1,BagLeKP3wgcP9EWu34Bd8w,mr-and-mrs-crab-orlando-2,Mr. & Mrs. Crab,https://s3-media1.fl.yelpcdn.com/bphoto/r-Gfjl...,False,https://www.yelp.com/biz/mr-and-mrs-crab-orlan...,90,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 28.460807, 'longitude': -81.3985764}",[delivery],$$,"{'address1': '7220 S Orange Blossom Trl', 'add...",+14072334945,(407) 233-4945,5071.954642
2,4S1WQMjW-aYkArmm6v-C2A,bar-harbor-seafood-orlando-2,Bar Harbor Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/ntfG2m...,False,https://www.yelp.com/biz/bar-harbor-seafood-or...,529,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 28.46023, 'longitude': -81.40509}",[delivery],$$,"{'address1': '2000 Premier Row', 'address2': '...",+14078514001,(407) 851-4001,5423.451060
3,4kEwYB1uCRKsajCxFHoOyA,hot-n-juicy-crawfish-orlando,Hot N Juicy Crawfish,https://s3-media2.fl.yelpcdn.com/bphoto/3iMa34...,False,https://www.yelp.com/biz/hot-n-juicy-crawfish-...,1454,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.0,"{'latitude': 28.4491703091685, 'longitude': -8...",[delivery],$$,"{'address1': '7572 W Sand Lake Rd', 'address2'...",+14073704655,(407) 370-4655,12158.498057
4,eaNpTadVytk8WoOM0Et2dg,the-boathouse-lake-buena-vista,The Boathouse,https://s3-media2.fl.yelpcdn.com/bphoto/zffi71...,False,https://www.yelp.com/biz/the-boathouse-lake-bu...,2889,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 28.37179431505456, 'longitude': -...",[delivery],$$$,"{'address1': '1620 E Buena Vista Dr', 'address...",+14079392628,(407) 939-2628,20012.406041


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,0RQLb6gLmruTjBeUZwIeoQ,i-drive-liquors-orlando,I-Drive Liquors,https://s3-media2.fl.yelpcdn.com/bphoto/vhtCNq...,False,https://www.yelp.com/biz/i-drive-liquors-orlan...,12,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",2.0,"{'latitude': 28.41999, 'longitude': -81.46139}","[pickup, delivery]",$$,"{'address1': '6169 Westwood Blvd', 'address2':...",+14073543730,(407) 354-3730,12366.311593
996,cplmWXCfPCn7EYaDzB2X3w,armandos-orlando-2,Armando's,https://s3-media3.fl.yelpcdn.com/bphoto/x4AdbP...,False,https://www.yelp.com/biz/armandos-orlando-2?ad...,96,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 28.410346, 'longitude': -81.242438}",[],$$,"{'address1': '10775 Narcoossee Rd', 'address2'...",+14072334741,(407) 233-4741,16670.854232
997,3QVcWJDtXEwhRXK-LXZg8w,hwy-441-diner-tavares-4,Hwy 441 Diner,https://s3-media3.fl.yelpcdn.com/bphoto/uHIa0D...,False,https://www.yelp.com/biz/hwy-441-diner-tavares...,74,"[{'alias': 'diners', 'title': 'Diners'}]",3.5,"{'latitude': 28.8179064034402, 'longitude': -8...",[delivery],$,"{'address1': '381 E Burleigh Blvd', 'address2'...",+13525085494,(352) 508-5494,48855.745579
998,IaGsMyZyyU-gtDnqg1NS4Q,cala-bella-shingle-creek-orlando-2,Cala Bella Shingle Creek,https://s3-media2.fl.yelpcdn.com/bphoto/b5JIhF...,False,https://www.yelp.com/biz/cala-bella-shingle-cr...,73,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 28.425776, 'longitude': -81.4349848}",[],$$$,"{'address1': '9939 Universal Blvd', 'address2'...",+14079963663,(407) 996-3663,10221.458701
999,byiM4huXJqSmKKrTTaOBAw,thai-jasmine-house-tavares,Thai Jasmine House,https://s3-media2.fl.yelpcdn.com/bphoto/Wm3ztR...,False,https://www.yelp.com/biz/thai-jasmine-house-ta...,99,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 28.8036022146513, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '1010 E Alfred St', 'address2': N...",+13527420956,(352) 742-0956,47203.013196


In [17]:
final_df.duplicated(subset='id').sum()

3

In [18]:
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [20]:
final_df.to_csv('Data/fina_results_FL_seafood.csv.gz',compression='gzip',
               index=False)